# rust

In [ ]:
// // test

open testing

()



## emit_expr

In [ ]:
inl emit_expr forall a t. (args : a) (code : string) : t =
    real
        $"Fable.Core.RustInterop.emitRustExpr !args !code" : t

()



In [ ]:
inl types () =
    global "[<Fable.Core.Erase; Fable.Core.Emit(\"Func0<$0>\")>] type Func0<'T> = class end"
    global "[<Fable.Core.Erase; Fable.Core.Emit(\"Func1<$0, $1>\")>] type Func0<'T, 'U> = class end"
    global "[<Fable.Core.Erase; Fable.Core.Emit(\"Box<$0>\")>] type Box<'T> = class end"
    global "[<Fable.Core.Erase; Fable.Core.Emit(\"dyn $0\")>] type Dyn<'T> = class end"
    global "[<Fable.Core.Erase; Fable.Core.Emit(\"Fn() -> $0\")>] type Fn<'T> = class end"
    global "[<Fable.Core.Erase; Fable.Core.Emit(\"Fn()\")>] type FnUnit = class end"
    global "[<Fable.Core.Erase; Fable.Core.Emit(\"FnOnce() -> $0\")>] type FnOnce<'T> = class end"
    global "[<Fable.Core.Erase; Fable.Core.Emit(\"Fn($0, $1)\")>] type ActionFn2<'T, 'U> = class end"
    global "[<Fable.Core.Erase; Fable.Core.Emit(\"impl $0\")>] type Impl<'T> = class end"
    global "[<Fable.Core.Erase; Fable.Core.Emit(\"mut $0\")>] type Mut<'T> = class end"
    global "[<Fable.Core.Erase; Fable.Core.Emit(\"&$0\")>] type Ref<'T> = class end"
    global "[<Fable.Core.Erase; Fable.Core.Emit(\"&'static $0\")>] type StaticRef<'T> = class end"
    global "[<Fable.Core.Erase; Fable.Core.Emit(\"MutCell<$0>\")>] type MutCell<'T> = class end"
    global "[<Fable.Core.Erase; Fable.Core.Emit(\"std::any::Any\")>] type std_any_Any = class end"
    global "[<Fable.Core.Erase; Fable.Core.Emit(\"std::cell::RefCell<$0>\")>] type std_cell_RefCell<'T> = class end"
    global "[<Fable.Core.Erase; Fable.Core.Emit(\"std::pin::Pin<$0>\")>] type std_pin_Pin<'T> = class end"
    global "[<Fable.Core.Erase; Fable.Core.Emit(\"std::rc::Rc<$0>\")>] type std_rc_Rc<'T> = class end"
    global "[<Fable.Core.Erase; Fable.Core.Emit(\"std::rc::Weak<$0>\")>] type std_rc_Weak<'T> = class end"
    global "[<Fable.Core.Erase; Fable.Core.Emit(\"std::sync::Arc<$0>\")>] type std_sync_Arc<'T> = class end"

()



In [ ]:
nominal ref_cell t = $"std_cell_RefCell<`t>"
nominal rc t = $"std_rc_Rc<`t>"
nominal weak_rc t = $"std_rc_Weak<`t>"
nominal box t = $"Box<`t>"
nominal mut_cell t = $"MutCell<`t>"
nominal pin t = $"std_pin_Pin<`t>"
nominal arc t = $"std_sync_Arc<`t>"
nominal dyn' t = $"Dyn<`t>"
nominal fn' t = $"Fn<`t>"
nominal action_fn2 t u = $"ActionFn2<`t, `u>"
nominal fn_once t = $"FnOnce<`t>"
nominal fn_unit = $"FnUnit"
nominal func0 t = $"Func0<`t>"
nominal func1 t u = $"Func0<`t, `u>"
nominal impl t = $"Impl<`t>"
nominal mut' t = $"Mut<`t>"
nominal ref' t = $"Ref<`t>"
nominal static_ref' t = $"StaticRef<`t>"

()



In [ ]:
inl (~!\) forall t. (code : string) : t =
    emit_expr () code

inl (~!\\) forall t u. ((args : t), (code : string)) : u =
    emit_expr args code

inl new_box forall t. (x : t) : box t =
    inl x = join x
    !\($'"Box::new(!x)"')

inl new_rc forall t. (x : t) : rc t =
    inl x = join x
    !\($'"std::rc::Rc::new(!x)"')

inl rc_clone forall t. (x : rc t) : rc t =
    inl x = join x
    !\($'"std::rc::Rc::clone(&!x)"')

inl rc_downgrade forall t. (x : rc t) : weak_rc t =
    inl x = join x
    !\($'"std::rc::Rc::downgrade(&!x)"')

inl new_ref_cell forall t. (x : t) : ref_cell t =
    inl x = join x
    !\($'"std::cell::RefCell::new(!x)"')

inl ref_cell_borrow forall t. (x : rc (ref_cell t)) : t =
    inl x = join x
    !\($'"*std::cell::RefCell::borrow(&std::rc::Rc::clone(&!x))"')

inl ref_cell_borrow_mut forall t. (x : rc (ref_cell t)) : mut' t =
    inl x = join x
    !\($'"*std::cell::RefCell::borrow_mut(&std::rc::Rc::clone(&!x))"')

inl to_mut forall t. (x : t) : mut' t =
    // !\($'"let mut !x = !x"')
    // !\($'"!x"')
    emit_expr () $"\"let mut !x = !x\""
    emit_expr () $"\"!x\""

inl ref_map forall t u. (fn : t -> u) (x : ref' t) : ref' u =
    !\($'"!fn(!x)"')

inl from_mut forall t. (x : mut' t) : t =
    !\($'"!x"')

inl new_arc forall t. (x : t) : arc t =
    inl x = join x
    !\($'"std::sync::Arc::new(!x)"')

inl box_fn forall t. (x : () -> ()) : box t =
    inl x = join x
    !\($'"Box::new(move || !x())"')

inl new_pin forall t. (x : t) : pin (box t) =
    inl x = join x
    !\($'"Box::pin(!x)"')

inl deref forall t. (ref : ref' t) : t =
    inl ref = join ref
    !\($'"*!ref"')

inl ops_deref forall t. (ref : t) : t =
    inl ref = join ref
    !\($'"core::ops::Deref::deref(&!ref)"')

inl func0_get forall t. (x : func0 t) : t =
    inl x = join x
    !\($'"!x()"')

inl func0_move forall t. (fn : func0 t) : t =
    inl fn = join fn
    !\($'"(move || !fn())()"')

inl move forall t. (fn : () -> t) : func0 t =
    inl fn = join fn
    !\($'"Func0::new(move || !fn())"')

inl to_static_ref_unbox forall t. (x : ref' t) : static_ref' t =
    $"!x |> unbox"

inl from_static_ref_unbox forall t. (x : static_ref' t) : ref' t =
    $"!x |> unbox"

inl box_leak forall t. (x : box t) : static_ref' (mut' t) =
    emit_expr () $"\"Box::leak(!x)\""


inl fix_closure depth' x' =
    inl depth = depth' |> fst
    if depth = 1
    then !\($'"!x' })"')
    elif depth = 2
    then !\($'"!x' }})"')
    elif depth = 3
    then !\($'"!x' }}})"')
    elif depth = 4
    then !\($'"!x' }}}})"')
    elif depth = 5
    then !\($'"!x' }}}}})"')
    elif depth = 6 // , 4) // ?
    then !\($'"!x' }}}}}})"')
    elif depth = 7 // , 5) // 7
    then !\($'"!x' }}}}}}})"')
    elif depth = 8 // , 5) // 7
    then !\($'"!x' }}}}}}}})"')

    inl depth = depth' |> snd
    if depth = 1
    then !\($'" { //"')
    elif depth = 2
    then !\($'" {{ //"')
    elif depth = 3
    then !\($'" {{{ //"')
    elif depth = 4
    then !\($'" {{{{ //"')
    elif depth = 5
    then !\($'" {{{{{ //"')
    elif depth = 6
    then !\($'"!x' {{{{{{ //"')
    elif depth = 7
    then !\($'"!x' {{{{{{{ //"')
    elif depth = 8
    then !\($'"!x' {{{{{{{{ //"')

()

